In [4]:
from util.utils import evaluate_imgs

In [5]:
vi_img_folder = "/home/2021/hexin/XJ2/D3FM/input/vi"
ir_img_folder = "/home/2021/hexin/XJ2/D3FM/input/ir"
f_img_folder = "/home/2021/hexin/XJ2/D3FM/output2/d3fm_mix/recon"
evaluate_imgs(f_img_folder,vi_img_folder,ir_img_folder,"RGB")

--------------------FLIR_03952.jpg--------------------
SSIM_1: 0.799, SSIM_2: 0.836
MI1: 0.927, MI2: 1.044
SD: 0.188
EN: 7.131
--------------------FLIR_04071.jpg--------------------
SSIM_1: 0.902, SSIM_2: 0.899
MI1: 0.779, MI2: 0.559
SD: 0.153
EN: 7.080
--------------------FLIR_04598.jpg--------------------
SSIM_1: 0.736, SSIM_2: 0.587
MI1: 1.156, MI2: 0.770
SD: 0.199
EN: 7.527
--------------------FLIR_08248.jpg--------------------
SSIM_1: 0.785, SSIM_2: 0.817
MI1: 0.442, MI2: 1.002
SD: 0.147
EN: 6.987


In [6]:
vi_img_folder = "/home/2021/hexin/XJ2/D3FM/input/vi"
ir_img_folder = "/home/2021/hexin/XJ2/D3FM/input/ir"
f_img_folder = "/home/2021/hexin/XJ2/D3FM/output2/recon"
evaluate_imgs(f_img_folder,vi_img_folder,ir_img_folder,"RGB")

--------------------FLIR_03952.jpg--------------------
SSIM_1: 0.799, SSIM_2: 0.836
MI1: 0.927, MI2: 1.044
SD: 0.188
EN: 7.131
--------------------FLIR_04071.jpg--------------------
SSIM_1: 0.902, SSIM_2: 0.899
MI1: 0.779, MI2: 0.559
SD: 0.153
EN: 7.080
--------------------FLIR_04598.jpg--------------------
SSIM_1: 0.736, SSIM_2: 0.587
MI1: 1.156, MI2: 0.770
SD: 0.199
EN: 7.527
--------------------FLIR_08248.jpg--------------------
SSIM_1: 0.785, SSIM_2: 0.817
MI1: 0.442, MI2: 1.002
SD: 0.147
EN: 6.987
